In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

任務1在地圖上標記所有發射地點。

In [4]:
# 定義發射地點的坐標
launch_sites = [
    {"name": "Launch Site 1", "coord": (28.573255, -80.646895)},
    {"name": "Launch Site 2", "coord": (28.563197, -80.576820)},
    {"name": "Launch Site 3", "coord": (28.562302, -80.577356)}
]

In [5]:
import folium

# 創建一個以第一個發射地點為中心的地圖
site_map = folium.Map(location=launch_sites[0]["coord"], zoom_start=10)

# 為每個發射地點添加標記
for site in launch_sites:
    folium.Marker(
        location=site["coord"],
        popup=site["name"],
        icon=folium.Icon(color='red', icon='rocket')
    ).add_to(site_map)

# 顯示地圖
site_map

In [6]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

In [7]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [8]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [9]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [10]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label

任務2 在地圖上標記每個網站的成功/失敗啟動。

In [11]:
# 定義發射地點的坐標和啟動狀態
launch_sites = [
    {"name": "Launch Site 1", "coord": (28.573255, -80.646895), "status": "success"},
    {"name": "Launch Site 2", "coord": (28.563197, -80.576820), "status": "failure"},
    {"name": "Launch Site 3", "coord": (28.562302, -80.577356), "status": "success"}
]

In [12]:
import folium

# 創建一個以第一個發射地點為中心的地圖
site_map = folium.Map(location=launch_sites[0]["coord"], zoom_start=10)

# 為每個發射地點添加標記
for site in launch_sites:
    # 根據啟動狀態選擇顏色
    color = 'green' if site["status"] == "success" else 'red'
    
    folium.Marker(
        location=site["coord"],
        popup=f"{site['name']} - {site['status'].capitalize()}",
        icon=folium.Icon(color=color, icon='rocket')
    ).add_to(site_map)

# 顯示地圖
site_map

In [13]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [14]:
marker_cluster = MarkerCluster()

In [15]:
import pandas as pd

# 假設 spacex_df 已經存在並包含 'class' 列
# 定義一個函數來根據 'class' 值返回顏色
def get_marker_color(launch_class):
    return 'green' if launch_class == 1 else 'red'

# 使用 apply 函數來創建 'marker_color' 列
spacex_df['marker_color'] = spacex_df['class'].apply(get_marker_color)

# 查看結果
print(spacex_df.head())

   Launch Site        Lat       Long  class marker_color
0  CCAFS LC-40  28.562302 -80.577356      0          red
1  CCAFS LC-40  28.562302 -80.577356      0          red
2  CCAFS LC-40  28.562302 -80.577356      0          red
3  CCAFS LC-40  28.562302 -80.577356      0          red
4  CCAFS LC-40  28.562302 -80.577356      0          red


In [17]:
import folium
from folium.plugins import MarkerCluster

# 創建一個以某個發射地點為中心的地圖
site_map = folium.Map(location=[28.573255, -80.646895], zoom_start=10)

# 創建 MarkerCluster
marker_cluster = MarkerCluster().add_to(site_map)

# 對於 spacex_df 資料框中的每一行，新增標記至 marker_cluster
for index, row in spacex_df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Long']],  # 假設 'Lat' 和 'Long' 是緯度和經度的列名
        popup=f"Launch Site: {row['Launch Site']}<br>Class: {row['class']}",
        icon=folium.Icon(color='white', icon_color=row['marker_color'])
    ).add_to(marker_cluster)

# 顯示地圖
site_map

任務3 計算發射場與其鄰近地區之間的距離。

In [18]:
# 發射場的坐標
launch_site_coord = (28.573255, -80.646895)

# 鄰近地區的坐標
nearest_city_coord = (28.3852, -81.5639)  # 這是一個示例坐標
nearest_rail_coord = (28.5721, -80.5853)  # 這是一個示例坐標
nearest_highway_coord = (28.5721, -80.6500)  # 這是一個示例坐標

In [19]:
import math

def haversine(lat1, lon1, lat2, lon2):
    # 將度數轉換為弧度
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    r = 6371  # 地球半徑（公里）
    return c * r

# 計算距離
city_distance = haversine(launch_site_coord[0], launch_site_coord[1], nearest_city_coord[0], nearest_city_coord[1])
rail_distance = haversine(launch_site_coord[0], launch_site_coord[1], nearest_rail_coord[0], nearest_rail_coord[1])
highway_distance = haversine(launch_site_coord[0], launch_site_coord[1], nearest_highway_coord[0], nearest_highway_coord[1])

print(f"Distance to nearest city: {city_distance:.2f} km")
print(f"Distance to nearest rail: {rail_distance:.2f} km")
print(f"Distance to nearest highway: {highway_distance:.2f} km")

Distance to nearest city: 92.03 km
Distance to nearest rail: 6.02 km
Distance to nearest highway: 0.33 km


In [20]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [21]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

TODO：使用 MousePosition 在最近的海岸線上標記一個點，並計算海岸線點與發射場之間的距離。

In [22]:
# 發射場的坐標
launch_site_coord = (28.573255, -80.646895)

# 最近海岸線的坐標
nearest_coastline_coord = (28.56367, -80.57163)  # 這是一個示例坐標

In [23]:
import math

def haversine(lat1, lon1, lat2, lon2):
    # 將度數轉換為弧度
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    r = 6371  # 地球半徑（公里）
    return c * r

# 計算距離
coastline_distance = haversine(launch_site_coord[0], launch_site_coord[1], nearest_coastline_coord[0], nearest_coastline_coord[1])

In [24]:
import folium
from folium.features import DivIcon

# 創建地圖
site_map = folium.Map(location=launch_site_coord, zoom_start=10)

# 在海岸線點上添加標記，顯示距離
distance_marker = folium.Marker(
    location=nearest_coastline_coord,
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>{:10.2f} KM</b></div>'.format(coastline_distance),
    )
).add_to(site_map)

# 顯示地圖
site_map

TODO在發射場與選定的海岸線點之間繪製多段線。

In [27]:
# 發射場的坐標
launch_site_coord = (28.573255, -80.646895)

# 最近海岸線的坐標
nearest_coastline_coord = (28.56367, -80.57163)

In [28]:
import folium

# 創建地圖
site_map = folium.Map(location=launch_site_coord, zoom_start=10)

# 定義多段線的坐標
coordinates = [launch_site_coord, nearest_coastline_coord]

# 創建多段線並添加到地圖
lines = folium.PolyLine(locations=coordinates, weight=1, color='blue')
site_map.add_child(lines)

# 顯示地圖
site_map

In [29]:
# 發射場的坐標
launch_site_coord = (28.573255, -80.646895)

# 鄰近地區的坐標
nearest_city_coord = (28.3852, -81.5639)  # 這是一個示例坐標
nearest_rail_coord = (28.5721, -80.5853)  # 這是一個示例坐標
nearest_highway_coord = (28.5721, -80.6500)  # 這是一個示例坐標

In [30]:
import math

def haversine(lat1, lon1, lat2, lon2):
    # 將度數轉換為弧度
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    r = 6371  # 地球半徑（公里）
    return c * r

# 計算距離
city_distance = haversine(launch_site_coord[0], launch_site_coord[1], nearest_city_coord[0], nearest_city_coord[1])
rail_distance = haversine(launch_site_coord[0], launch_site_coord[1], nearest_rail_coord[0], nearest_rail_coord[1])
highway_distance = haversine(launch_site_coord[0], launch_site_coord[1], nearest_highway_coord[0], nearest_highway_coord[1])

In [31]:
import folium
from folium.features import DivIcon

# 創建地圖
site_map = folium.Map(location=launch_site_coord, zoom_start=10)

# 定義標記和線的函數
def add_marker_and_line(coord, distance, label):
    # 添加標記
    folium.Marker(
        location=coord,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12; color:#d35400;"><b>{:10.2f} KM</b></div>'.format(distance),
        )
    ).add_to(site_map)

    # 添加多段線
    folium.PolyLine(locations=[launch_site_coord, coord], weight=1, color='blue').add_to(site_map)

# 添加到最近城市、鐵路和高速公路的標記和線
add_marker_and_line(nearest_city_coord, city_distance, "City")
add_marker_and_line(nearest_rail_coord, rail_distance, "Rail")
add_marker_and_line(nearest_highway_coord, highway_distance, "Highway")

# 顯示地圖
site_map